## Housing Price Predictor
- Utilizes a dual model approach to predict the pricing of housing. 
- Created using data from H. Ahmed E. and Moustafa M. (2016). House Price Estimation from Visual and Textual Features.In Proceedings of the 8th International Joint Conference on Computational Intelligence (IJCCI 2016)ISBN 978-989-758-201-1, pages 62-68. DOI: 10.5220/0006040700620068


## Helper Methods
- Tools we may use later for data pre-processing and graphing

In [1]:
from collections.abc import Sequence
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
import csv
import numpy as np
import sys
from sklearn.metrics import roc_curve, auc, confusion_matrix
from helper_methods import *

%matplotlib inline

import cv2
import glob
from IPython.display import Image


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=45)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

# Plot an ROC. pred - the predictions, y - the expected output.
def plot_roc(pred,y, file_path):
    fpr, tpr, thresholds = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.savefig(file_path)
    plt.close()

## Set up Environment
Create test output folders and define paths 

In [2]:
base_path=os.path.join(os.getcwd(), 'test-output/')

iteration='iteration-3'
full_path = os.path.join(base_path, iteration)
try:
        os.mkdir(base_path)
except Exception as e:
     print(f"Probably safe to ignore the following error: \n{e}")
try:
    os.mkdir(full_path)
except Exception as e:
    print(f"{e}\nExiting to protect previous work.")
    sys.exit(0)
    

Probably safe to ignore the following error: 
[WinError 183] Cannot create a file when that file already exists: 'c:\\Users\\timef\\Documents\\Workspaces\\Python\\csc180\\housing-price-predictor\\test-output/'


## Data Extraction
- code involved with the extraction of the dataset

In [3]:
#import statements
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from pathlib import Path
import os
import sys
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt 


#reads text file data into dataframe (essentially a table)
TXT_DATASET_LOCATION = "./Houses Dataset/HousesInfo.txt"

cols=["Bedrooms","Bathrooms","area","zipcode","price"]

df = pd.read_csv(TXT_DATASET_LOCATION, sep= " ", header=None, names=cols)
# z-score encode and one-hot encode the features. 'price' is our output feature. 
encode_text_dummy(df, 'zipcode')
encode_numeric_zscore(df, 'Bedrooms')
encode_numeric_zscore(df, 'Bathrooms')
encode_numeric_zscore(df, 'area')


# retrieve 

#df.head()

#

# How do I read from a text file

In [4]:
#retrieval of images into arrays. 

## Image Handling
- Next we extract the images from the dataset and concatenate the 4 images in the dataset together

In [5]:
## Bathroom.jpg
new_images=[]
HOUSE_DATASET_LOCATION = "./Houses dataset/" 
for number in range(1, 536):
    for path in glob.glob("./Houses dataset/" + str(number) + "_bathroom.jpg"):
        if os.path.isfile(path):
            new_images.append(path) 

new_images

# creates variable to store all the images into appropriate columns
img= pd.DataFrame(new_images,columns = ['bathroom_img'])

In [6]:
## bedroom images
bedroom_images = []
for number in range(1, 536):
    for path in glob.glob(HOUSE_DATASET_LOCATION + str(number) + "_bedroom.jpg"):
        if os.path.isfile(path):
            bedroom_images.append(path) 

frontal_images = []
for number in range(1, 536):
    for path in glob.glob(HOUSE_DATASET_LOCATION + str(number) + "_frontal.jpg"):
        if os.path.isfile(path):
            frontal_images.append(path) 

kitchen_images = []
for number in range(1, 536):
    for path in glob.glob(HOUSE_DATASET_LOCATION + str(number) + "_kitchen.jpg"):
        if os.path.isfile(path):
            kitchen_images.append(path) 

img['bedroom_img']=bedroom_images
img['frontal_img']=frontal_images
img['kitchen_img']=kitchen_images
img.head()

,bathroom_img,bedroom_img,frontal_img,kitchen_img
0,./Houses dataset/1_bathroom.jpg,./Houses dataset/1_bedroom.jpg,./Houses dataset/1_frontal.jpg,./Houses dataset/1_kitchen.jpg
1,./Houses dataset/2_bathroom.jpg,./Houses dataset/2_bedroom.jpg,./Houses dataset/2_frontal.jpg,./Houses dataset/2_kitchen.jpg
2,./Houses dataset/3_bathroom.jpg,./Houses dataset/3_bedroom.jpg,./Houses dataset/3_frontal.jpg,./Houses dataset/3_kitchen.jpg
3,./Houses dataset/4_bathroom.jpg,./Houses dataset/4_bedroom.jpg,./Houses dataset/4_frontal.jpg,./Houses dataset/4_kitchen.jpg
4,./Houses dataset/5_bathroom.jpg,./Houses dataset/5_bedroom.jpg,./Houses dataset/5_frontal.jpg,./Houses dataset/5_kitchen.jpg


In [7]:
# Code which concatenates houses images into one image for each house
images_output=[]
for row_index,row in img.iterrows():
            inputImages=[]
            outputImage = np.zeros((128, 128, 3), dtype="uint8")
            image_temp1 = cv2.imread(row.bathroom_img)
            image1 = cv2.resize(image_temp1, (64 , 64))
            
            image_temp2 = cv2.imread(row.bedroom_img)
            image2 = cv2.resize(image_temp2, (64 , 64))
            
            image_temp3 = cv2.imread(row.frontal_img)
            image3 = cv2.resize(image_temp3, (64 , 64))
            
            image_temp4 = cv2.imread(row.kitchen_img)
            image4 = cv2.resize(image_temp4, (64 , 64))
              
            inputImages.append(image1)
            inputImages.append(image2)
            inputImages.append(image3)
            inputImages.append(image4)
            
            outputImage[0:64, 0:64] = inputImages[0]
            outputImage[0:64, 64:128] = inputImages[1]
            outputImage[64:128, 64:128] = inputImages[2]
            outputImage[64:128, 0:64] = inputImages[3]
            
            #uncomment this if you want to see a boatload of images that it is concatenating
            images_output.append(outputImage)      
img_array = np.asarray(images_output)
img_array = img_array.astype("float32") / 255

#### Do split


In [8]:
x, y = to_xy(df, 'price')
x_train, x_test,img_train, img_test, y_train, y_test = train_test_split(x,img_array, y, test_size=0.20, random_state=42)

### Create Model
The model is comprised of two smaller models. One model is for processing images. THis one is a CNN. The other model is for processing the data about the house. This is a FCN. 

In [10]:
checkpointer = ModelCheckpoint(filepath=os.path.join(full_path, "best_weights.keras"), verbose=0, save_best_only=True) # save best model
for i in range(5):
    print(f"Training Model {i}")
    # Housing Data FCNN

    data_fcn_visible = Input(shape=(x_train.shape[1],))
    hidden1=Dense(32, activation='relu')(data_fcn_visible)
    hidden2 = Dense(16, activation='relu')(hidden1)

    # Image CNN
    image_cnn_visible = Input(shape=(128, 128, 3))
    conv1 = Conv2D(32, kernel_size=(3,3), activation='relu')(image_cnn_visible)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    flat = Flatten()(pool1)

    merge = concatenate([hidden1, flat])
    # interpretation model
    dense1=(Dense(16, activation='relu'))(flat)
    dense2 = Dense(16, activation='relu')(dense1)
    outputs=(Dense(1))(dense2)
    model = Model(inputs=[data_fcn_visible, image_cnn_visible], outputs=outputs)
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='min', restore_best_weights=True)
    model.compile(loss='mean_squared_error', optimizer='adam') #not done here, change as see fit (is broken)
    history = model.fit([x_train, img_train], y_train, validation_data=([x_test, img_test], y_test), epochs=100, callbacks=[checkpointer, monitor])
    plot_losses(history=history, base_path=full_path, iteration=i)




Training Model 0
Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - loss: 3.1335 - val_loss: 0.2281
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - loss: 0.3549 - val_loss: 0.0820
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - loss: 0.0457 - val_loss: 0.0413
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - loss: 0.0195 - val_loss: 0.0047
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0057 - val_loss: 0.0058
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 0.0046 - val_loss: 0.0033
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0039 - val_loss: 0.0037
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - loss: 0.0036 - val_loss: 0.0033
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - loss: 0.0035 - val_loss: 0.0032
Epoch 10/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 11/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 0.0033 - val_loss: 0.0031
Epoch 11: early stopping
Restoring

### Load, Predict, Evaluate

In [ ]:
model.load_weights(os.path.join(full_path, 'best_weights.keras'))
prediction = model.predict([x_test, img_test])
print(prediction.shape)
pred_roc = prediction[:,0]
y_true = np.argmax(y_test, axis=1)

plot_roc(pred_roc, y_true, os.path.join(full_path, 'cnn-roc-curve.png'))
try:
        pred = np.argmax(prediction, axis=1)
        # confusion matrix
        cm = confusion_matrix(y_true, pred)
        plt.figure()
        plot_confusion_matrix(cm, ['normal', 'malicious'])
        plt.savefig(os.path.join(full_path, 'cnn-confusion-matrix.png'))
        plt.close()
except Exception as e:
        print(f"error creating confusion matrix plot:\n{e} ")
try:
        with open(os.path.join(full_path, 'cnn-metrics.txt'), 'x') as file:
                prediction=np.argmax(prediction, axis=1)
                y_true = np.argmax(y_test, axis=1)
                accuracy_score = metrics.accuracy_score(y_true, prediction)
                precision_score = metrics.precision_score(y_true, prediction, average='weighted')
                recall_score = metrics.recall_score(y_true, prediction, average= "weighted")
                fl_score = metrics.f1_score(y_true, prediction, average= "weighted")
                file.write(f"Accuracy Score: {accuracy_score}\n")
                file.write(f"Precision Score: {precision_score}\n")
                file.write("Recall score: {}\n".format(recall_score))
                file.write("F1 score: {}\n".format(fl_score))
                log_loss = metrics.log_loss(y_test, prediction)
                print(f"Log Loss: {log_loss}")                
                file.write(str(metrics.classification_report(y_true, prediction)))
                file.write("\nNumpy array of predictions\n")
                file.write(np.array_str(prediction[0:5]))
                file.write("y_test:\n")
                file.write(np.array2string(y_test[0:5]))
except OSError as e:
        print(f"Error while writing model metrics: \n{e}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
(107, 1)


IndexError: index 1 is out of bounds for axis 1 with size 1